Link: https://www.tensorflow.org/get_started/tflearn

In [14]:
#Importing the training and test set
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import urllib
import urllib.request

import tensorflow as tf
import numpy as np

IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

if not os.path.exists(IRIS_TRAINING):
    raw = urllib.request.urlopen(IRIS_TRAINING_URL).read().decode()
    with open(IRIS_TRAINING,'w') as f:
        f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = urllib.request.urlopen(IRIS_TEST_URL).read().decode()
    with open(IRIS_TEST,'w') as f:
        f.write(raw)

To make this work, I had to change urllib.urlopen to urllib.request.urlopen because I use Python3. https://stackoverflow.com/questions/39975367/attributeerror-module-urllib-has-no-attribute-urlopen 

Also added .decode() according to https://stackoverflow.com/questions/43783954/tf-contrib-learn-load-csv-with-header-not-working-in-tensorflow-1-1

In [15]:
#Load Datasets
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

 tf.contrib.learn has many predefined models, called **estimators**. Here we use a DNN classifier.

In [16]:
# Specify that all features have real-value data (of dimension 4)
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[10, 20, 10],
                                            n_classes=3,
                                            model_dir="/tmp/iris_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_tf_random_seed': None, '_task_id': 0, '_save_checkpoints_steps': None, '_master': '', '_evaluation_master': '', '_environment': 'local', '_save_summary_steps': 100, '_model_dir': '/tmp/iris_model', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_num_worker_replicas': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000000A368C50>, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_session_config': None, '_is_chief': True}


In [17]:
#Input pipeline
def get_train_inputs():
    x = tf.constant(training_set.data)
    y = tf.constant(training_set.target)
    return x, y

In [18]:
#Fitting the classifier
classifier.fit(input_fn=get_train_inputs, steps = 2000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/iris_model\model.ckpt.
INFO:tensorflow:step = 1, loss = 1.53682
INFO:tensorflow:global_step/sec: 1201.92
INFO:tensorflow:step = 101, loss = 0.157632 (0.086 sec)
INFO:tensorflow:global_step/sec: 1366.12
INFO:tensorflow:step = 201, loss = 0.0824196 (0.072 sec)
INFO:tensorflow:global_step/sec: 1424.5
INFO:tensorflow:step = 301, loss = 0.0679037 (0.071 sec)
INFO:tensorflow:global_step/sec: 1538.46
INFO:tensorflow:step = 401, loss = 0.0619661 (0.065 sec)
INFO:tensorflow:global_step/sec: 1492.54
INFO:tensorflow:step = 501, loss = 0.0585318 (0.068 sec)
INFO:tensorflow:global_s

DNNClassifier(params={'feature_columns': (_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None),), 'hidden_units': [10, 20, 10], 'activation_fn': <function relu at 0x00000000076187B8>, 'gradient_clip_norm': None, 'input_layer_min_slice_size': None, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x000000000A3687F0>, 'dropout': None, 'embedding_lr_multipliers': None, 'optimizer': None})

In [19]:
#Getting test inputs
def get_test_inputs():
    x = tf.constant(test_set.data)
    y = tf.constant(test_set.target)
    return x, y
#Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=get_test_inputs,
                                     steps=1)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-06-30-15:10:49
INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-2000
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-06-30-15:10:50
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.966667, global_step = 2000, loss = 0.0706575

Test Accuracy: 0.966667



In [23]:
# Classify two new flower samples.
def new_samples():
    return np.array(
    [[6.4, 3.2, 4.5, 1.5],
     [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)

predictions = list(classifier.predict(input_fn=new_samples))

print(
    "New Samples, Class Predictions:    {}\n"
    .format(predictions))

#Getting confidence levels
probabilities = list(classifier.predict_proba(input_fn=new_samples))

print(probabilities)

Instructions for updating:
Please switch to predict_classes, or set `outputs` argument.
INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-2000
New Samples, Class Predictions:    [1, 2]

INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-2000
[array([  1.14029826e-05,   9.99914169e-01,   7.43451528e-05], dtype=float32), array([  2.06771772e-04,   3.41850191e-01,   6.57943070e-01], dtype=float32)]
